In [5]:
# Step-by-step Implementation for Greenwashing Detection

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load the provided Excel file
data_path = 'AIB Group_Example.xlsx'
claims_df = pd.read_excel(data_path, sheet_name='Claims')

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Traceback (most recent call last):
  File "c:\Users\bumin\anaconda3\envs\c20599\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
# Hypothetical user comments (this would typically be scraped from social media)
user_comments = [
    "Great initiative! I believe they're genuinely committed.",
    "This feels like greenwashing to me.",
    "Are they really making an impact or just marketing?",
    "Impressive steps towards sustainability!",
    "Sounds good on paper, but I doubt their real commitment.",
]

# Initialize pre-trained models
sentiment_model = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:

# Analyze sentiment of user comments
sentiments = sentiment_model(user_comments)

# Convert sentiments to DataFrame
sentiment_df = pd.DataFrame(sentiments)
sentiment_df['comment'] = user_comments

# Compute embeddings for corporate claims and user comments
claims_embeddings = embedder.encode(claims_df['Description'].tolist())
comments_embeddings = embedder.encode(user_comments)

# Calculate semantic similarity between claims and comments
similarity_matrix = cosine_similarity(claims_embeddings, comments_embeddings)

In [ ]:
# Create a contradiction score based on negative sentiment and low similarity
# Lower similarity and negative sentiment indicate potential contradiction
contradiction_scores = []
for i, comment in enumerate(user_comments):
    sentiment = sentiment_df.iloc[i]['label']
    sentiment_score = sentiment_df.iloc[i]['score']
    avg_similarity = np.mean(similarity_matrix[:, i])

    # Define contradiction score
    if sentiment == 'NEGATIVE':
        contradiction_score = (1 - avg_similarity) * sentiment_score
    else:
        contradiction_score = 0  # Positive sentiment implies no contradiction

    contradiction_scores.append(contradiction_score)

sentiment_df['contradiction_score'] = contradiction_scores

In [ ]:
# Summarize results
print("Sentiment and Contradiction Analysis:")
print(sentiment_df[['comment', 'label', 'score', 'contradiction_score']])